

```
 Name: Muhammad Saad
 CodaLab Username: Saad_33_B
 Rank: 30
```
 Solution to codalab's SentiMix Hindi-English competition: 
 [SentiMix Hindi-English]( https://competitions.codalab.org/competitions/20654)

Train +Test Set After Cleaning:
 [Google Drive Link](https://drive.google.com/drive/folders/1hNnbsghCTVpsskq5KpHbTJSVJqOhubsZ?usp=sharing)

Pretrained Model:
[Google Drive Link](https://drive.google.com/file/d/1xmCegUpnMwCsoquCYYBqkxkKXf5m9wtA/view?usp=sharing)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset

#Defining Paths

In [ ]:
#Defining Paths

#Directories
MOUNT_DIR = '/content/drive/'
DRIVE_DIR = MOUNT_DIR+'MyDrive/'
SENTIMIX_DIR = DRIVE_DIR+'sentimix-hindi/'

#Train and Test Data Files
TRAIN_SET_FILE = SENTIMIX_DIR + 'train_set_without_usernames.json'
TEST_SET_FILE = SENTIMIX_DIR + 'test_set_without_usernames.json'

#BERT Model 
MODEL_NAME = 'DeepPavlov/bert-base-multilingual-cased-sentence'

# Connecting to Google Drive

In [ ]:
#Connecting with google drive
from google.colab import drive

drive.mount(MOUNT_DIR)

# Loading Datasets

In [ ]:
import json

#Loading our train dataset
with open(TRAIN_SET_FILE) as file:
  train_data = json.load(file)
  
#Loading our test dataset
with open(TEST_SET_FILE) as file:
  test_data = json.load(file)

train_set = dict(train_data)
test_set = dict(test_data)
del train_data
del test_data

In [ ]:
import pandas as pd

train_df = pd.DataFrame(train_set).T
test_df = pd.DataFrame(test_set).T

train_df['sentence'] = train_df['sentence'].apply(lambda sen: " ".join(sen).strip())
test_df['sentence'] = test_df['sentence'].apply(lambda sen: " ".join(sen).strip())  


In [ ]:
train_df.head()

In [ ]:
#Exploring our data
train_df['sentiment'].value_counts()

In [ ]:
#Encoding sentiments using integers
labels = dict()
for idx, lab in enumerate(train_df.sentiment.unique()):
  labels[lab] = idx

train_df['sentiment'] = train_df.sentiment.replace(labels)
test_df['sentiment'] = test_df.sentiment.replace(labels)

labels

#Loading Tokenizers

In [ ]:
#Tokenizer 
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)


#For Training
encoded_data_train = tokenizer.batch_encode_plus(
    train_df.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=128, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']

attention_masks_train = encoded_data_train['attention_mask']

labels_train = torch.tensor(train_df.sentiment.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

#For Validation

encoded_data_val = tokenizer.batch_encode_plus(
    test_df.sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True,
    max_length=128, 
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']

attention_masks_val = encoded_data_val['attention_mask']

labels_val = torch.tensor(test_df.sentiment.values)

dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# Loading Pretrained BERT Model

In [ ]:
model = BertForSequenceClassification.from_pretrained(MODEL_NAME,
                                                      num_labels=len(labels),
                                                      output_attentions=True,
                                                      output_hidden_states=False)


#Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32 

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

#Creating Optimizers and Schedulers

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup, Adafactor
                  
optimizer = Adafactor(
    model.parameters(),
    lr=1e-5,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=int(0.125*len(dataloader_train)),
                                            num_training_steps=len(dataloader_train)*epochs)



# Performance Metrics

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, true_labels):

    label_dict_inverse = {v: k for k, v in labels.items()}
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = true_labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Fine Tuning

In [ ]:
def evaluate(dataloader_val,model):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
import random

seed_val = 33
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

####
# Uncomment the line below to load a previous saved model
#model.load_state_dict(torch.load(f'{SENTIMIX_DIR}sentimix_model.model', map_location=torch.device(device)))
###
model.to(device)

    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'{SENTIMIX_DIR}augmented_sentimix_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation,model)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

# Evaluation

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_eval = BertForSequenceClassification.from_pretrained(MODEL_NAME,
                                                      num_labels=len(labels),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model_eval.to(device)

model_eval.load_state_dict(torch.load(f'{SENTIMIX_DIR}sentimix_model.model', map_location=torch.device('cpu')))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/bert-base-multilingual-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

## Classification Report

In [23]:
_, predictions, true_vals = evaluate(dataloader_validation,model_eval)
accuracy_per_class(predictions, true_vals)
print(f'F1 Score: {f1_score_func(predictions, true_vals)}')

Class: negative
Accuracy: 685/900

Class: positive
Accuracy: 813/1000

Class: neutral
Accuracy: 637/1100

F1 Score: 0.7085907587775198


In [24]:
from sklearn.metrics import classification_report
print(f'Labels: {labels}')
print(classification_report(true_vals,np.argmax(predictions, axis=1).flatten()))

Labels: {'negative': 0, 'positive': 1, 'neutral': 2}
              precision    recall  f1-score   support

           0       0.70      0.76      0.73       900
           1       0.77      0.81      0.79      1000
           2       0.66      0.58      0.62      1100

    accuracy                           0.71      3000
   macro avg       0.71      0.72      0.71      3000
weighted avg       0.71      0.71      0.71      3000



## Creating answers.txt

In [ ]:
with open(TEST_SET_FILE) as file:
  test_dataset = json.load(file)

test_dataset = dict(test_dataset)

keys_list = [key for key in test_dataset.keys()]
pred_list = np.argmax(predictions, axis=1).flatten()
label_dict_inverse = {v: k for k, v in labels.items()}

with open(f'{SENTIMIX_DIR}answer.txt','w') as file:
    file.write('Uid,Sentiment\n')
    for key,pred in zip(keys_list,pred_list):
        file.write(f'{key},{label_dict_inverse[pred]}\n')

del test_dataset


# Credits


[Multi Class Text Classification With Deep Learning Using BERT](https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613)
